In [5]:
import sys
sys.path.insert(0, "/notebooks/pipenv")
sys.path.insert(0, "/notebooks/nebula3_database")
from torch.nn.functional import softmax
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch
import random
import csv
import numpy as np
import pickle

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
device = "cuda:0"
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def load_stories(dataset = 'roc'):
    if dataset ==  'roc':
        stories = []
        with open("1000ROC.csv") as file:
            tsv_file = csv.reader(file, delimiter=",")
            for line in tsv_file:
                #print(line)
                line.pop(0)
                #print(line)
                line.pop(0)
                stories.append(line)
    return(stories)

In [15]:
def create_stories(stories):
    stories_with_candidates = []
    for story in stories:
        story_length = len(story) - 1
        corpus_length = len(stories) -1
        scenes = []
        #print(orig_story[0])
        for sentence in story:
            candidates = []
            candidates.append(sentence)
            for candidate in range(0, random.randint(3, 10 )):
                cand_sent = random.randint(0, story_length)
                cand_story = random.randint(0, corpus_length)
                candidates.append(stories[cand_story][cand_sent])
            scenes.append(candidates)
        stories_with_candidates.append(scenes)
    return(stories_with_candidates)

In [13]:
def story_compatability(scene1, scene2):
    rows_ = []
    for sent_a in scene1:
        cols_ = []
        for sent_b in scene2:
            encoded = tokenizer.encode_plus(sent_a, sent_b, return_tensors='pt').to(device)
            seq_relationship_logits = model(**encoded)[0]
            probs = softmax(seq_relationship_logits, dim=1)
            score = probs[0][0].tolist()
            cols_.append(score)
        rows_.append(cols_)
    return(np.array(rows_))

In [16]:
stories = load_stories('roc')
candidated_stories = create_stories(stories)
stories_with_scores = []
for story in candidated_stories:
    scenes_scores = []
    for idx in range(0, len(story) -1):
        scene1 = story[idx]
        scene2 = story[idx + 1]
        scene_matrix = story_compatability(scene1, scene2)
        scenes_scores.append(scene_matrix)
    storie_with_scores = {
        'story': story,
        'scores': scenes_scores
    }

    stories_with_scores.append(storie_with_scores)
    
with open('roc1k.pickle', 'wb') as handle:
    pickle.dump(stories_with_scores, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [19]:
with open('roc1k.pickle', 'wb') as handle:
    pickle.dump(stories_with_scores, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
for i in stories_with_scores:
    print(i)